# Knowledge Bases for Amazon Bedrock - Exemplo de ponta a ponta

Este notebook fornece código de exemplo para construir um índice vazio do OpenSearch Serverless (OSS), uma Knowledge Base do Amazon Bedrock e ingerir documentos no índice.

Um pipeline de dados que ingere documentos (normalmente armazenados no Amazon S3) em uma base de conhecimento, ou seja, um banco de dados vetorial como o OpenSearch Service Serverless (OSS), para que esteja disponível para pesquisa quando uma pergunta for recebida.

#### Etapas:
- Criar uma execution role do Amazon Bedrock Knowledge Base com as políticas necessárias para acessar dados do S3 e gravar embeddings no OSS.
- Criar um índice vazio do OpenSearch serverless.
- Baixar documentos.
- Criar uma Knowledge Base do Amazon Bedrock.
- Criar uma fonte de dados dentro da Knowledge Base que se conectará ao Amazon S3.
- Iniciar um trabalho de ingestão usando as APIs do Knowledge Bases (KB) que lerão dados do S3, dividirão em chunks, converterão os chunks em embeddings usando o modelo Amazon Titan Embeddings e armazenarão esses embeddings no OSS. Tudo isso sem precisar construir, implantar e gerenciar o pipeline de dados.

## Caso de uso
### Conjunto de dados
Neste exemplo, você usará vários anos das Cartas aos Acionistas da Amazon como um corpus de texto para realizar perguntas e respostas.

## Setup 
Antes de executar o restante deste notebook, você precisará executar as células abaixo para garantir que as bibliotecas necessárias estejam instaladas e também se conectar ao Amazon Bedrock.

In [ ]:
%pip install -U opensearch-py==2.3.1
%pip install -U boto3==1.33.2
%pip install -U retrying==1.3.4

In [8]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [48]:
import json
import os
import boto3
import pprint
import time
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from utility import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss
import random
from retrying import retry
from urllib.request import urlretrieve

suffix = random.randrange(200, 900)

boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_client = boto3_session.client("bedrock-runtime", region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent", region_name=region_name)
aoss_client = boto3_session.client("opensearchserverless")
s3_client = boto3_session.client("s3")

bucket_name = "cdbes01e08-sancho" #troque o nome deste bucket por um bucket seu!

pp = pprint.PrettyPrinter(indent=2)

In [49]:
# Definindo a pergunta que faremos ao modelo
query = "What is Amazon's doing in the field of generative AI?"

In [50]:
# Definindo o modelo que será utilizando neste notebook
model_id = "anthropic.claude-v2" # para claude v2, utilize "anthropic.claude-v2 or claude-instant-v1"
model_arn = f"arn:aws:bedrock:us-east-1::foundation-model/{model_id}"

In [51]:
# Fazendo a pergunta antes de criar a base de conhecimento e sem utilizar RAG
key_word_args = {
  "modelId": model_id,
  "body": "{\"prompt\":\"Human: " + query + "\\nAssistant:\",\"max_tokens_to_sample\":300,\"temperature\":1,\"top_k\":250,\"top_p\":0.999,\"stop_sequences\":[\"\\n\\nHuman:\"],\"anthropic_version\":\"bedrock-2023-05-31\"}"
}

response = bedrock_client.invoke_model(**key_word_args)
response_body = json.loads(response.get("body").read())
generated_text = response_body.get("completion")

pp.pprint(generated_text)

(' Amazon has made some notable investments and developments related to '
 'generative AI:\n'
 '\n'
 '- AWS OpenAI Service - In 2021, Amazon announced a new fully managed AWS '
 'service that allows access to OpenAI models like GPT-3 for text generation. '
 'This makes advanced generative AI more accessible to developers.\n'
 '\n'
 '- Alexa Conversations - The Alexa virtual assistant uses some generative '
 'techniques to have more natural conversations. The Alexa Skills Kit enables '
 'developers to build conversational Alexa skills.\n'
 '\n'
 '- Amazon Kendra - This is a natural language search service that uses AI to '
 'understand user queries and find answers in documents. It generates natural '
 'language responses. \n'
 '\n'
 '- Amazon CodeWhisperer - Announced in 2022, this is a code generation '
 'service to help developers by generating code based on simple natural '
 'language prompts. It uses GPT-3 and other techniques under the hood.\n'
 '\n'
 '- Amazon has research scient

## Criar um repositório de vetores - índice no Amazon OpenSearch Serverless

### Passo 1 - Criar role de execução do Bedrock NB, poíticas e coleção no OSS
Primeiro de tudo, temos que criar um repositório de vetores. Nesta seção, vamos usar *Amazon OpenSearch Serverless.*

O Amazon OpenSearch Serverless é uma opção serverless do Amazon OpenSearch Service. Como desenvolvedor, você pode usar o OpenSearch Serverless para executar cargas de trabalho de petabytes sem precisar configurar, gerenciar e dimensionar clusters do OpenSearch. Você obtém os mesmos tempos de resposta interativos de milissegundos do OpenSearch Service com a simplicidade de um ambiente serverless. Pague apenas pelo que usar, escalando recursos automaticamente para fornecer a capacidade certa para sua aplicação, sem impactar a ingestão de dados.

In [52]:
bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket_name)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role["Role"]["Arn"]

In [53]:
# Cria as políticas de segurança, rede e acesso a dados
vector_store_name = f"bedrock-sample-rag-{suffix}"
encryption_policy, network_policy, access_policy = create_policies_in_oss(
    vector_store_name=vector_store_name,
    aoss_client=aoss_client,
    bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn)

In [16]:
# Cria a coleção no OpenSearch Serverless
collection = aoss_client.create_collection(name=vector_store_name,type="VECTORSEARCH")

In [17]:
pp.pprint(collection)
time.sleep(10)

{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '314',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Thu, 22 Aug 2024 00:54:10 '
                                                 'GMT',
                                         'x-amzn-requestid': '240888d9-1c5c-4a80-987b-480cd8b9bab9'},
                        'HTTPStatusCode': 200,
                        'RequestId': '240888d9-1c5c-4a80-987b-480cd8b9bab9',
                        'RetryAttempts': 0},
  'createCollectionDetail': { 'arn': 'arn:aws:aoss:us-east-1:381491840841:collection/x24qujh1lmt7yh6ha6pb',
                              'createdDate': 1724288050518,
                              'id': 'x24qujh1lmt7yh6ha6pb',
                              'kmsKeyArn': 'auto',
                              'lastModifiedDate': 1724288050518,
                             

In [18]:
collection_id = collection["createCollectionDetail"]["id"]
host = collection_id + "." + region_name + ".aoss.amazonaws.com"
print(host)

x24qujh1lmt7yh6ha6pb.us-east-1.aoss.amazonaws.com


In [19]:
# Cria a política oss e atacha isso na role de execução do Amazon Bedrock
create_oss_policy_attach_bedrock_execution_role(collection_id=collection_id,
                                                bedrock_kb_execution_role=bedrock_kb_execution_role)

Opensearch serverless arn:  arn:aws:iam::381491840841:policy/AmazonBedrockOSSPolicyForKnowledgeBase_688


## Passo 2 - Cria o vector index

In [29]:
credentials = boto3_session.get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region_name, "aoss")

index_name = f"bedrock-sample-index-{suffix}"
body_json = {
   "settings": {
      "index.knn": "true"
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "FAISS",
            "dimension": 1536
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"         }
      }
   }
}

oss_client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

## Pode levar até um minuto para que as regras de acesso aos dados sejam aplicadas
time.sleep(60)

In [30]:
# Cria o índice
# Tive que alterar o tipo de VECTOR para FAISS, então crie manualmente o Index.
response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))
print("\nCriando o índice:")
print(response)

RequestError: RequestError(400, 'mapper_parsing_exception', 'OpenSearch exception [type=mapper_parsing_exception, reason=No handler for type [FAISS] declared on field [vector]]- server : [envoy]')

## Download dos dados

In [25]:
!mkdir -p data

urls = [
    "https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf",
    "https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf",
    "https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf",
    "https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf"
]

filenames = [
    "AMZN-2022-Shareholder-Letter.pdf",
    "AMZN-2021-Shareholder-Letter.pdf",
    "AMZN-2020-Shareholder-Letter.pdf",
    "AMZN-2019-Shareholder-Letter.pdf"
]

data_directory = "\\data\\"

for idx, url in enumerate(urls):
    file_path = data_directory + filenames[idx]
    urlretrieve(url, file_path)


J� existe uma subpasta ou um arquivo data.
Erro ao processar: data.


FileNotFoundError: [Errno 2] No such file or directory: '\\data\\AMZN-2022-Shareholder-Letter.pdf'

#### Sobe os dados no bucket do S3

In [ ]:
def upload_directory(path,bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                s3_client.upload_file(os.path.join(root,file),bucket_name,file)

upload_directory(data_directory, bucket_name)

## Criar a base de conhecimento
Passos:
- Inicialize a configuração do OpenSearch serverless, que incluirá o ARN da coleção.
- Inicialize a estratégia de divisão em chunks, com base na qual o KB dividirá os documentos em pedaços do tamanho igual ao tamanho de chunk mencionado em `chunkingStrategyConfiguration`.
- Inicialize a configuração do S3, que será usada para criar o objeto de fonte de dados posteriormente.
- Inicialize o ARN do modelo de embeddings Titan, pois este será usado para criar os embeddings para cada um dos chunks de texto.

In [26]:
opensearchServerlessConfiguration = {
            "collectionArn": collection["createCollectionDetail"]["arn"],
            "vectorIndexName": index_name,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}",
}

embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1"

name = f"bedrock-sample-knowledge-base-{suffix}"
description = "Amazon shareholder letter knowledge base."
roleArn = bedrock_kb_execution_role_arn


Forneça as configurações acima como entrada para o método `create_knowledge_base`, que criará a base de conhecimento.

In [27]:
# Cria a base de conhecimento
@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = name,
        description = description,
        roleArn = roleArn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

In [31]:
try:
    kb = create_knowledge_base()
except Exception as err:
    print(f"{err=}, {type(err)=}")

In [32]:
pp.pprint(kb)

{ 'createdAt': datetime.datetime(2024, 8, 22, 1, 59, 47, 515624, tzinfo=tzutc()),
  'description': 'Amazon shareholder letter knowledge base.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:381491840841:knowledge-base/XRRUN1PPJ0',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v1'}},
  'knowledgeBaseId': 'XRRUN1PPJ0',
  'name': 'bedrock-sample-knowledge-base-414',
  'roleArn': 'arn:aws:iam::381491840841:role/AmazonBedrockExecutionRoleForKnowledgeBase_688',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { 'collectionArn': 'arn:aws:aoss:us-east-1:381491840841:collection/x24qujh1lmt7yh6ha6pb',
                                                                   'fieldMapping': { 'metadataField': 'text-metadata',
                                                                       

In [33]:
# Recupera a Knowledge Base
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb["knowledgeBaseId"])

Agora precisamos criar uma fonte de dados, que será associada à base de conhecimento criada acima. Assim que a fonte de dados estiver pronta, poderemos começar a ingerir os documentos.

In [34]:
# Cria a fonte de dados na base de conhecimento
create_ds_response = bedrock_agent_client.create_data_source(
    name = name,
    description = description,
    knowledgeBaseId = kb["knowledgeBaseId"],
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration":s3Configuration
    },
    vectorIngestionConfiguration = {
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)
ds = create_ds_response["dataSource"]
pp.pprint(ds)

{ 'createdAt': datetime.datetime(2024, 8, 22, 2, 0, 7, 503494, tzinfo=tzutc()),
  'dataSourceConfiguration': { 's3Configuration': { 'bucketArn': 'arn:aws:s3:::cdbes01e08-sancho'},
                               'type': 'S3'},
  'dataSourceId': '2JEBE7LOJP',
  'description': 'Amazon shareholder letter knowledge base.',
  'knowledgeBaseId': 'XRRUN1PPJ0',
  'name': 'bedrock-sample-knowledge-base-414',
  'status': 'AVAILABLE',
  'updatedAt': datetime.datetime(2024, 8, 22, 2, 0, 7, 503494, tzinfo=tzutc()),
  'vectorIngestionConfiguration': { 'chunkingConfiguration': { 'chunkingStrategy': 'FIXED_SIZE',
                                                               'fixedSizeChunkingConfiguration': { 'maxTokens': 512,
                                                                                                   'overlapPercentage': 20}}}}


In [35]:
# Recupera a fonte de dados
bedrock_agent_client.get_data_source(knowledgeBaseId = kb["knowledgeBaseId"], dataSourceId = ds["dataSourceId"])

{'ResponseMetadata': {'RequestId': '47fd0050-5d9a-4fdd-be5b-f33b36a29f23',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 22 Aug 2024 02:00:14 GMT',
   'content-type': 'application/json',
   'content-length': '587',
   'connection': 'keep-alive',
   'x-amzn-requestid': '47fd0050-5d9a-4fdd-be5b-f33b36a29f23',
   'x-amz-apigw-id': 'c41DXF2kIAMEhIA=',
   'x-amzn-trace-id': 'Root=1-66c69bae-22de823f1216664e66c498b5'},
  'RetryAttempts': 0},
 'dataSource': {'knowledgeBaseId': 'XRRUN1PPJ0',
  'dataSourceId': '2JEBE7LOJP',
  'name': 'bedrock-sample-knowledge-base-414',
  'status': 'AVAILABLE',
  'description': 'Amazon shareholder letter knowledge base.',
  'dataSourceConfiguration': {'type': 'S3',
   's3Configuration': {'bucketArn': 'arn:aws:s3:::cdbes01e08-sancho'}},
  'vectorIngestionConfiguration': {'chunkingConfiguration': {'chunkingStrategy': 'FIXED_SIZE',
    'fixedSizeChunkingConfiguration': {'maxTokens': 512,
     'overlapPercentage': 20}}},
  'createdAt': datetime.datetime(

### Inicia o JOB de ingestão
Assim que a base de conhecimento e a fonte de dados forem criadas, podemos iniciar o job de ingestão.

Durante o job de ingestão, a base de conhecimento buscará os documentos na fonte de dados, pré-processará para extrair o texto, dividirá em chunks com base no tamanho de chunk fornecido, criará embeddings para cada chunk e então gravará no banco de dados vetorial, neste caso o Amazon OpenSearch Serverless.

In [36]:
# Inicia o job de ingestão
start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb["knowledgeBaseId"], 
                                                              dataSourceId = ds["dataSourceId"])

In [37]:
job = start_job_response["ingestionJob"]


In [38]:
pp.pprint(job)

{ 'dataSourceId': '2JEBE7LOJP',
  'ingestionJobId': 'ZRUVVYJMGV',
  'knowledgeBaseId': 'XRRUN1PPJ0',
  'startedAt': datetime.datetime(2024, 8, 22, 2, 0, 21, 669046, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2024, 8, 22, 2, 0, 21, 669046, tzinfo=tzutc())}


In [39]:
# Recupera o job job e aguarda seu término 
while(job["status"]!="COMPLETE" ):
  get_job_response = bedrock_agent_client.get_ingestion_job(
      knowledgeBaseId = kb["knowledgeBaseId"],
        dataSourceId = ds["dataSourceId"],
        ingestionJobId = job["ingestionJobId"]
  )
  job = get_job_response["ingestionJob"]
pp.pprint(job)
time.sleep(40)

{ 'dataSourceId': '2JEBE7LOJP',
  'ingestionJobId': 'ZRUVVYJMGV',
  'knowledgeBaseId': 'XRRUN1PPJ0',
  'startedAt': datetime.datetime(2024, 8, 22, 2, 0, 21, 669046, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 4,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 4},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2024, 8, 22, 2, 0, 38, 77570, tzinfo=tzutc())}


In [40]:
kb_id = kb["knowledgeBaseId"]
pp.pprint(kb_id)

'XRRUN1PPJ0'


## Testando a base de conhecimento
### Usando a API RetrieveAndGenerate
Por baixo do capô, a API RetrieveAndGenerate converte consultas em embeddings, pesquisa a base de conhecimento e então enriquece o prompt do modelo fundacional com os resultados da pesquisa como informações de contexto e retorna a resposta gerada pelo FM à pergunta. Para conversas com múltiplos turnos, o KB gerencia a memória de curto prazo da conversa para fornecer resultados mais contextuais.

A saída da API RetrieveAndGenerate inclui a resposta gerada, atribuição da fonte, bem como os chunks de texto recuperados.

In [41]:
# utilizando a Knowledge Base utilizando a API RetrieveAndGenerate
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region_name)

In [42]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": kb_id,
            "modelArn": model_arn
        }
    },
)

generated_text = response["output"]["text"]
pp.pprint(generated_text)

('Amazon has been working on developing its own large language models (LLMs) '
 'for a while. The company believes LLMs and generative AI will transform and '
 'improve customer experiences across its consumer, seller, brand, and creator '
 'platforms. Additionally, Amazon is making LLMs more accessible through AWS '
 'by offering machine learning chips like Trainium and Inferentia to help '
 'companies of all sizes train and run their models cost-effectively.')


In [43]:
## Imprime a atribuição da fonte/citações dos documentos originais para ver se a resposta gerada pertence ao contexto.
citations = response["citations"]
contexts = []
for citation in citations:
    retrievedReferences = citation["retrievedReferences"]
    for reference in retrievedReferences:
        contexts.append(reference["content"]["text"])

pp.pprint(contexts)

[ 'Amazon has been using machine learning extensively for 25 years, employing '
  'it in everything from personalized ecommerce recommendations, to '
  'fulfillment center pick paths, to drones for Prime Air, to Alexa, to the '
  'many machine learning services AWS offers (where AWS has the broadest '
  'machine learning functionality and customer base of any cloud provider). '
  'More recently, a newer form of machine learning, called Generative AI, has '
  'burst onto the scene and promises to significantly accelerate machine '
  'learning adoption. Generative AI is based on very Large Language Models '
  '(trained on up to hundreds of billions of parameters, and growing), across '
  'expansive datasets, and has radically general and broad recall and learning '
  'capabilities. We have been working on our own LLMs for a while now, believe '
  'it will transform and improve virtually every customer experience, and will '
  'continue to invest substantially in these models across all o

### Retrieve API
A API Retrieve converte consultas de usuários em embeddings, pesquisa a base de conhecimento e retorna os resultados relevantes, dando a você mais controle para construir fluxos de trabalho personalizados com base nos resultados da pesquisa semântica.

A saída da API Retrieve inclui os chunks de texto recuperados, o tipo de localização e URI dos dados de origem, bem como as pontuações de relevância das recuperações.

In [44]:
# API de recuperação para buscar apenas o contexto relevante.
relevant_documents = bedrock_agent_runtime_client.retrieve(
    retrievalQuery= {
        "text": query
    },
    knowledgeBaseId=kb_id,
    retrievalConfiguration= {
        "vectorSearchConfiguration": {
            "numberOfResults": 3 #  irá recuperar os 3 principais documentos que correspondem à consulta.
        }
    }
)

In [45]:
pp.pprint(relevant_documents["retrievalResults"])

[ { 'content': { 'text': 'Amazon has been using machine learning extensively '
                         'for 25 years, employing it in everything from '
                         'personalized ecommerce recommendations, to '
                         'fulfillment center pick paths, to drones for Prime '
                         'Air, to Alexa, to the many machine learning services '
                         'AWS offers (where AWS has the broadest machine '
                         'learning functionality and customer base of any '
                         'cloud provider). More recently, a newer form of '
                         'machine learning, called Generative AI, has burst '
                         'onto the scene and promises to significantly '
                         'accelerate machine learning adoption. Generative AI '
                         'is based on very Large Language Models (trained on '
                         'up to hundreds of billions of parameters, and '
     

## Limpando a casa
Lembre-se de excluir todos os recursos criados, pois você incorrerá em custos para armazenar documentos no índice OSS.

In [46]:
# Deleta a base de conhecimento
bedrock_agent_client.delete_data_source(dataSourceId = ds["dataSourceId"], knowledgeBaseId=kb["knowledgeBaseId"])
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb["knowledgeBaseId"])

oss_client.indices.delete(index=index_name)

aoss_client.delete_collection(id=collection_id)

aoss_client.delete_access_policy(type="data", name=access_policy["accessPolicyDetail"]["name"])
aoss_client.delete_security_policy(type="network", name=network_policy["securityPolicyDetail"]["name"])
aoss_client.delete_security_policy(type="encryption", name=encryption_policy["securityPolicyDetail"]["name"])

{'ResponseMetadata': {'RequestId': 'c482d5fc-f23b-4f08-91e5-615bf70dcda5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c482d5fc-f23b-4f08-91e5-615bf70dcda5',
   'date': 'Thu, 22 Aug 2024 02:57:33 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [47]:
# Exclui a role e as políticas
from utility import delete_iam_role_and_policies
delete_iam_role_and_policies()

0